# Import Dependencies

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.sklearn

# Custom functions from .py files
from web_scrape import get_transcript
from preprocess import clean, preprocess

# Import/Preprocess Data

In [2]:
# Web scrape most recent speeches using our imported 'get_transcript'
goodyear_az = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-goodyear-az-october-28')
bullhead_az = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-bullhead-city-az-october-28')
omaha_ne = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-omaha-ne-october-27')
wsalem_wi = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-west-salem-wisconsin-october-27')
lansing_mi = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lansing-michigan-october-27')
martinsburg_pa = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-martinsburg-pa-october-26')
lititz_pa = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lititz-pa-october-26')
allentown_pa = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-allentown-pa-october-26')

# Create corpus of all speeches
corpus = [goodyear_az, bullhead_az, omaha_ne, wsalem_wi, lansing_mi, martinsburg_pa, lititz_pa, allentown_pa]

# Create corpus for each state
az = [goodyear_az, bullhead_az]
ne = [omaha_ne]
wi = [wsalem_wi]
mi = [lansing_mi]
pa = [martinsburg_pa, lititz_pa, allentown_pa]

# Clean corpus using our imported 'clean' function
clean_corpus = clean(corpus)

# Clean state corpuses
clean_az = clean(az)
clean_ne = clean(ne)
clean_wi = clean(wi)
clean_mi = clean(mi)
clean_pa = clean(pa)

# Preprocess the corpus using our imported 'preprocess' function
processed_corpus = preprocess(clean_corpus)

# Preprocess our state corpuses
processed_az = preprocess(clean_az)
processed_ne = preprocess(clean_ne)
processed_wi = preprocess(clean_wi)
processed_mi = preprocess(clean_mi)
processed_pa = preprocess(clean_pa)

# Doc-Term Matrix

In [3]:
# Countvectorizer, make dataframes

cv = CountVectorizer(stop_words='english')
cv_az = CountVectorizer(stop_words='english')
cv_ne = CountVectorizer(stop_words='english')
cv_wi = CountVectorizer(stop_words='english')
cv_mi = CountVectorizer(stop_words='english')
cv_pa = CountVectorizer(stop_words='english')

X = cv.fit_transform(processed_corpus)
df = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

X_az = cv_az.fit_transform(processed_az)
df_az = pd.DataFrame(X_az.toarray(), columns=cv_az.get_feature_names())

X_ne = cv_ne.fit_transform(processed_ne)
df_ne = pd.DataFrame(X_ne.toarray(), columns=cv_ne.get_feature_names())

X_wi = cv_wi.fit_transform(processed_wi)
df_wi = pd.DataFrame(X_wi.toarray(), columns=cv_wi.get_feature_names())

X_mi = cv_mi.fit_transform(processed_mi)
df_mi = pd.DataFrame(X_mi.toarray(), columns=cv_mi.get_feature_names())

X_pa = cv_pa.fit_transform(processed_pa)
df_pa = pd.DataFrame(X_pa.toarray(), columns=cv_pa.get_feature_names())

# LDA

In [5]:
lda = LatentDirichletAllocation(n_components=9, random_state=0)
lda.fit(X)

lda_az = LatentDirichletAllocation(n_components=9, random_state=0)
lda_az.fit(X_az)

lda_ne = LatentDirichletAllocation(n_components=9, random_state=0)
lda_ne.fit(X_ne)

lda_wi = LatentDirichletAllocation(n_components=9, random_state=0)
lda_wi.fit(X_wi)

lda_mi = LatentDirichletAllocation(n_components=9, random_state=0)
lda_mi.fit(X_mi)

lda_pa = LatentDirichletAllocation(n_components=9, random_state=0)
lda_pa.fit(X_pa)

LatentDirichletAllocation(n_components=9, random_state=0)

In [15]:
# LDA Visualization for all speeches
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, X, cv)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.025769 -0.119887       1        1  23.913761
1     -0.110481 -0.160104       2        1  20.578552
0      0.218876  0.015173       3        1  20.010533
4     -0.006017  0.131133       4        1  19.461169
2     -0.128147  0.133684       5        1  16.035986, topic_info=           Term        Freq       Total Category  logprob  loglift
1122      right   92.000000   92.000000  Default  30.0000  30.0000
546       great  107.000000  107.000000  Default  29.0000  29.0000
1496       year   60.000000   60.000000  Default  28.0000  28.0000
774        love   60.000000   60.000000  Default  27.0000  27.0000
1463        win   67.000000   67.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
1006  president   11.278172   43.777305   Topic5  -4.5957   0.4741
466       fight    7.404131   19.693698   Topic5  -5.0166   0.8521
68      arizona    7.208410   27.055082   Topic5  -5.0434   0.5077
1434       vote    7.862706   49.340367   Topic5  -4.9565  -0.0063
120         big    5.993136   38.372282   Topic5  -5.2280  -0.0264

[274 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         2  0.735599            able
19        1  0.768403  administration
19        2  0.109772  administration
19        3  0.109772  administration
27        1  0.722344         african
...     ...       ...             ...
1483      3  0.819571           worry
1486      3  0.819571             wow
1495      3  0.737307            yeah
1496      3  0.985365            year
1499      5  0.954533       yesterday

[381 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 5, 3])